<a href="https://colab.research.google.com/github/AlekhyaGangopadhyay/IITPatna_Internship_LLMs/blob/main/Qwen2_VL_7B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================================
#  Qwen2-VL-7B  ▸  Finance Harassment/Abuse Video Dataset
#  Full Pipeline: Train → Infer → Accuracy Score
#  Dataset : iamalekhya/Finance_set  (318 videos, 9 columns)
#  Run on  : Google Colab  →  Runtime ▸ T4 / A100 GPU
# ============================================================

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 1 ── Install (run once, then restart runtime)
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 1 — Run this, then Runtime > Restart Session, then run from Cell 2
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes
!pip install rouge-score qwen-vl-utils openpyxl opencv-python scikit-learn
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 2 ── Mount Google Drive
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
from google.colab import drive
drive.mount('/content/drive')

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 3 ── ⚙️  CONFIG  — only edit this section
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# ── Dataset ───────────────────────────────────────────────────
HF_DATASET     = "iamalekhya/Finance_set"
EXCEL_FALLBACK = "/content/drive/MyDrive/IITP_Internship/Finance_set.xlsx"

# ── BUG FIX 1: VIDEO_DIR must be a LIST, not a tuple of strings ──
# Your original code had:
#   VIDEO_DIR = "path1","path2","path3"   ← Python makes this a TUPLE, os.path.join() crashes
# Fixed: proper Python list
VIDEO_DIRS = [
    "/content/drive/My Drive/IITP_Internship/AG",
    "/content/drive/My Drive/IITP_Internship/ag",
    "/content/drive/My Drive/IITP_Internship/Alekhya",
    "/content/drive/My Drive/IITP_Internship/Pritam",
    "/content/drive/My Drive/IITP_Internship/Videos",
]
# NOTE: "Shared with me" drives mount as "Shared drives" in Colab.
# If videos still not found, run this in a cell to check exact path:
#   import os; print(os.listdir('/content/drive/Shared drives/'))
VIDEO_EXT = ".mp4"

# ── Training ──────────────────────────────────────────────────
NUM_FRAMES  = 8
MAX_SEQ_LEN = 2048
EPOCHS      = 3
BATCH_SIZE  = 1
GRAD_ACCUM  = 8
LORA_RANK   = 16
EVAL_SPLIT  = 0.10
RANDOM_SEED = 42

# ── Paths ─────────────────────────────────────────────────────
OUTPUT_DIR          = "./qwen2vl_finance"
ADAPTER_DRIVE       = "/content/drive/MyDrive/IIT_Internship/qwen2vl_finance_adapter"
RESULTS_EXCEL_DRIVE = "/content/drive/MyDrive/IIT_Internship/finance_eval_results.xlsx"

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 4 ── Imports
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
import os, re, json, cv2, shutil, torch, warnings
import numpy as np
import pandas as pd
from PIL import Image
from datasets import Dataset, load_dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from rouge_score import rouge_scorer as rs
from unsloth import FastVisionModel
from trl import SFTTrainer, SFTConfig

warnings.filterwarnings("ignore")
print("✅ Imports OK")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 5 ── Load Dataset
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
print("Loading dataset...")
try:
    ds = load_dataset(HF_DATASET, split="train")
    df = ds.to_pandas()
    print(f"✅ Loaded from HuggingFace: {len(df)} rows")
except Exception as e:
    print(f"⚠️  HF load failed ({e}), falling back to Excel...")
    df = pd.read_excel(EXCEL_FALLBACK)
    print(f"✅ Loaded from Excel: {len(df)} rows")

# Normalise column names
df.columns = [c.strip() for c in df.columns]
print("   Columns:", df.columns.tolist())

REQUIRED = ["uni_id", "Category", "Aspect", "Intent", "Explanation", "Country", "Source", "Sector"]
missing  = [c for c in REQUIRED if c not in df.columns]
if missing:
    raise ValueError(f"Missing columns: {missing}  — actual: {df.columns.tolist()}")

df = df.dropna(subset=REQUIRED).reset_index(drop=True)
print(f"   Clean rows : {len(df)}")
print(f"   Categories : {df['Category'].unique().tolist()}")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 6 ── Prompt Templates
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
ALL_ASPECTS = sorted(df["Aspect"].unique().tolist())

SYSTEM_PROMPT = f"""You are an expert video analyst specialising in financial harassment, fraud, abuse, and workplace misconduct.

Given video frames, output a single valid JSON object with EXACTLY these fields:

{{
  "category"   : "Harassment" | "Bullying",
  "aspect"     : one of the known aspect labels (see list below),
  "intent"     : 1 (intentional) | 0 (unintentional),
  "explanation": "1-2 sentence description of what is happening and why it is problematic",
  "country"    : "country name or Unknown",
  "source"     : "YouTube | News | Training | Documentary | Other",
  "sector"     : "Finance | Healthcare | Education | Retail | Tech | Government | General"
}}

Known aspect labels (pick the closest match):
{json.dumps(ALL_ASPECTS, indent=2)}

Rules:
- Output ONLY the JSON. No preamble, no markdown, no extra text.
- Use double quotes for all strings.
- intent must be an integer (1 or 0), not a string."""

USER_PROMPT = """These frames are sampled evenly from a video about financial misconduct or harassment.

Carefully observe:
• Who is involved and what is happening
• The type and form of misconduct
• Whether the act is intentional
• The country/industry context

Return your analysis as a JSON object only."""

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 7 ── Helpers
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

def find_video(uni_id: str) -> str:
    """
    Search across all VIDEO_DIRS for a file named uni_id.mp4 (case-insensitive).
    Returns full path if found, else empty string.
    """
    filename = f"{uni_id}{VIDEO_EXT}"
    for folder in VIDEO_DIRS:
        path = os.path.join(folder, filename)
        if os.path.exists(path):
            return path
    # Try case-insensitive search in each folder
    for folder in VIDEO_DIRS:
        if not os.path.isdir(folder):
            continue
        for f in os.listdir(folder):
            if f.lower() == filename.lower():
                return os.path.join(folder, f)
    return ""


def extract_frames(video_path: str, n: int = NUM_FRAMES) -> list:
    """Return n evenly-spaced PIL frames from a video file."""
    if not video_path or not os.path.exists(video_path):
        return []
    cap   = cv2.VideoCapture(video_path)
    total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if total == 0:
        cap.release()
        return []
    frames = []
    for idx in np.linspace(0, total - 1, n, dtype=int):
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ok, frame = cap.read()
        if ok:
            frames.append(Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)))
    cap.release()
    return frames


def row_to_json(row) -> str:
    return json.dumps({
        "category"   : str(row["Category"]).strip(),
        "aspect"     : str(row["Aspect"]).strip(),
        "intent"     : int(row["Intent"]),
        "explanation": str(row["Explanation"]).strip(),
        "country"    : str(row["Country"]).strip(),
        "source"     : str(row["Source"]).strip(),
        "sector"     : str(row["Sector"]).strip(),
    }, indent=2)


def parse_json_response(text: str) -> dict:
    try:
        m = re.search(r'\{.*\}', text, re.DOTALL)
        if m:
            return json.loads(m.group())
    except Exception:
        pass
    return {}

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 8 ── Load Model + LoRA
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
print("\nLoading Qwen2-VL-7B...")
model, tokenizer = FastVisionModel.from_pretrained(
    model_name                 = "unsloth/Qwen2-VL-7B-Instruct",
    load_in_4bit               = True,
    use_gradient_checkpointing = "unsloth",
)
model = FastVisionModel.get_peft_model(
    model,
    finetune_vision_layers     = True,
    finetune_language_layers   = True,
    finetune_attention_modules = True,
    finetune_mlp_modules       = True,
    r            = LORA_RANK,
    lora_alpha   = LORA_RANK,
    lora_dropout = 0,
    bias         = "none",
    random_state = RANDOM_SEED,
)
print("✅ Model + LoRA ready")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# REPLACE YOUR CELL 9 + CELL 10 WITH THIS
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# ── CELL 9 ── Build Train / Eval splits ──────────────────────
train_df, eval_df = train_test_split(df, test_size=EVAL_SPLIT, random_state=RANDOM_SEED)
train_df = train_df.reset_index(drop=True)
eval_df  = eval_df.reset_index(drop=True)
print(f"\nSplit → Train: {len(train_df)}  |  Eval: {len(eval_df)}")


def build_samples(dataframe, split_name="train"):
    """Build raw samples list — text + PIL images kept together."""
    samples, skipped = [], 0

    for i, row in dataframe.iterrows():
        uni_id     = str(row["uni_id"]).strip()
        video_path = find_video(uni_id)
        frames     = extract_frames(video_path)

        if not frames:
            print(f"  ⚠️  [{split_name}] Skipping {uni_id} — video not found")
            skipped += 1
            continue

        target = row_to_json(row)

        user_content = [{"type": "image", "image": f} for f in frames]
        user_content.append({"type": "text", "text": USER_PROMPT})

        messages = [
            {"role": "system",    "content": [{"type": "text", "text": SYSTEM_PROMPT}]},
            {"role": "user",      "content": user_content},
            {"role": "assistant", "content": [{"type": "text", "text": target}]},
        ]

        text = tokenizer.apply_chat_template(
            messages, tokenize=False, add_generation_prompt=False
        )

        samples.append({
            "text"       : text,
            "images"     : frames,
            "uni_id"     : uni_id,
            "target_json": target,
        })

        if (i + 1) % 30 == 0:
            print(f"  [{split_name}] {i+1}/{len(dataframe)}  skipped={skipped}")

    print(f"  ✅ [{split_name}] ready={len(samples)}  skipped={skipped}")
    return samples


print("\n📦 Building training samples...")
train_samples = build_samples(train_df, "train")

print("\n📦 Building eval samples...")
eval_samples  = build_samples(eval_df,  "eval")

print(f"\n✅ Total usable → Train: {len(train_samples)}  Eval: {len(eval_samples)}")


# ── CELL 10 ── Custom Collator + Train ───────────────────────
from torch.utils.data import Dataset as TorchDataset
from torch.nn.utils.rnn import pad_sequence

class VideoQADataset(TorchDataset):
    """
    Tokenises each sample on-the-fly, passing PIL images to the
    processor so pixel_values are created correctly.
    """
    def __init__(self, samples, tokenizer, max_length=2048):
        self.samples    = samples
        self.tokenizer  = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        s      = self.samples[idx]
        text   = s["text"]
        images = s["images"]   # list of PIL Images

        # Tokenise text + images together
        encoding = self.tokenizer(
            text,
            images          = images,
            return_tensors  = "pt",
            max_length      = self.max_length,
            truncation      = True,
            padding         = False,
        )

        # Squeeze batch dim added by return_tensors="pt"
        item = {k: v.squeeze(0) for k, v in encoding.items()}

        # Labels = input_ids (causal LM); mask padding with -100
        labels = item["input_ids"].clone()
        labels[labels == self.tokenizer.pad_token_id] = -100
        item["labels"] = labels

        return item


def collate_fn(batch):
    """Pad a batch of variable-length tokenised samples."""
    pad_id = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else 0

    input_ids      = pad_sequence([b["input_ids"]      for b in batch], batch_first=True, padding_value=pad_id)
    attention_mask = pad_sequence([b["attention_mask"] for b in batch], batch_first=True, padding_value=0)
    labels         = pad_sequence([b["labels"]         for b in batch], batch_first=True, padding_value=-100)

    result = {
        "input_ids"     : input_ids,
        "attention_mask": attention_mask,
        "labels"        : labels,
    }

    # pixel_values may or may not be present depending on processor version
    if "pixel_values" in batch[0]:
        result["pixel_values"] = torch.cat([b["pixel_values"].unsqueeze(0) for b in batch], dim=0)

    # image_grid_thw is used by Qwen2-VL to know frame layout
    if "image_grid_thw" in batch[0]:
        result["image_grid_thw"] = torch.cat([b["image_grid_thw"] for b in batch], dim=0)

    return result


# Build PyTorch datasets
train_torch_dataset = VideoQADataset(train_samples, tokenizer, max_length=MAX_SEQ_LEN)

FastVisionModel.for_training(model)

from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir                  = OUTPUT_DIR,
    per_device_train_batch_size = BATCH_SIZE,
    gradient_accumulation_steps = GRAD_ACCUM,
    num_train_epochs            = EPOCHS,
    learning_rate               = 2e-4,
    fp16                        = not torch.cuda.is_bf16_supported(),
    bf16                        = torch.cuda.is_bf16_supported(),
    logging_steps               = 5,
    save_steps                  = 100,
    save_total_limit            = 2,
    warmup_ratio                = 0.03,
    lr_scheduler_type           = "cosine",
    optim                       = "adamw_8bit",
    seed                        = RANDOM_SEED,
    dataloader_pin_memory       = False,   # avoids CUDA pin_memory issues with PIL
    remove_unused_columns       = False,
    report_to                   = "none",
)

trainer = Trainer(
    model         = model,
    args          = training_args,
    train_dataset = train_torch_dataset,
    data_collator = collate_fn,
)

print("\n🚀 Training started...")
trainer.train()
print("✅ Training complete!")

# Save adapter
import shutil, os
model.save_pretrained("qwen2vl_finance_adapter")
tokenizer.save_pretrained("qwen2vl_finance_adapter")
os.makedirs(ADAPTER_DRIVE, exist_ok=True)
shutil.copytree("qwen2vl_finance_adapter", ADAPTER_DRIVE, dirs_exist_ok=True)
print(f"✅ Adapter saved → {ADAPTER_DRIVE}")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 11 ── Inference helper
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
FastVisionModel.for_inference(model)

def predict(frames: list) -> dict:
    """Run model on a list of PIL frames; return parsed dict."""
    # BUG FIX 2: type must be "image" not "video" — Qwen2-VL takes individual frames as images
    user_content = [{"type": "image", "image": f} for f in frames]
    user_content.append({"type": "text", "text": USER_PROMPT})

    messages = [
        {"role": "system", "content": [{"type": "text", "text": SYSTEM_PROMPT}]},
        {"role": "user",   "content": user_content},
    ]

    text_in = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    inputs = tokenizer(text_in, return_tensors="pt").to("cuda")

    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens = 512,
            temperature    = 0.1,
            do_sample      = False,
        )

    decoded = tokenizer.decode(out[0], skip_special_tokens=True)
    return parse_json_response(decoded)

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 12 ── Run inference on FULL dataset (train + eval)
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
print("\n" + "="*65)
print("  🔍 Running inference on ALL samples (train + eval)")
print("="*65)

all_samples = train_samples + eval_samples
all_results = []

for i, s in enumerate(all_samples):
    # BUG FIX 3: s["videos"] → s["images"]  (key was named "images" when built above)
    pred = predict(s["images"])
    tgt  = json.loads(s["target_json"])

    all_results.append({
        "uni_id"          : s["uni_id"],
        "split"           : "train" if i < len(train_samples) else "eval",
        # ground truth
        "true_category"   : tgt.get("category",    ""),
        "true_aspect"     : tgt.get("aspect",       ""),
        "true_intent"     : str(tgt.get("intent",   "")),
        "true_explanation": tgt.get("explanation",  ""),
        "true_country"    : tgt.get("country",      ""),
        "true_source"     : tgt.get("source",       ""),
        "true_sector"     : tgt.get("sector",       ""),
        # predictions
        "pred_category"   : str(pred.get("category",    "")).strip(),
        "pred_aspect"     : str(pred.get("aspect",       "")).strip(),
        "pred_intent"     : str(pred.get("intent",       "")).strip(),
        "pred_explanation": str(pred.get("explanation",  "")).strip(),
        "pred_country"    : str(pred.get("country",      "")).strip(),
        "pred_source"     : str(pred.get("source",       "")).strip(),
        "pred_sector"     : str(pred.get("sector",       "")).strip(),
    })

    if (i + 1) % 20 == 0:
        print(f"  Inferred {i+1}/{len(all_samples)}")

print(f"✅ Inference done for {len(all_results)} samples")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 13 ── Accuracy Scoring
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
scorer = rs.RougeScorer(["rougeL"], use_stemmer=True)

def score_split(rows, label="ALL"):
    if not rows:
        return {}

    def acc(t, p):
        return accuracy_score(
            [x.lower().strip() for x in t],
            [x.lower().strip() for x in p]
        )

    tc  = [r["true_category"]    for r in rows]
    pc  = [r["pred_category"]    for r in rows]
    ta  = [r["true_aspect"]      for r in rows]
    pa  = [r["pred_aspect"]      for r in rows]
    ti  = [r["true_intent"]      for r in rows]
    pi  = [r["pred_intent"]      for r in rows]
    tn  = [r["true_country"]     for r in rows]
    pn  = [r["pred_country"]     for r in rows]
    ts  = [r["true_sector"]      for r in rows]
    ps  = [r["pred_sector"]      for r in rows]
    tsr = [r["true_source"]      for r in rows]
    psr = [r["pred_source"]      for r in rows]

    rouge_scores = [
        scorer.score(r["true_explanation"], r["pred_explanation"])["rougeL"].fmeasure
        for r in rows
    ]
    avg_rouge = float(np.mean(rouge_scores))

    cat_acc = acc(tc, pc)
    asp_acc = acc(ta, pa)
    int_acc = acc(ti, pi)
    cty_acc = acc(tn, pn)
    sec_acc = acc(ts, ps)
    src_acc = acc(tsr, psr)

    overall = (
        cat_acc   * 0.25 +
        asp_acc   * 0.25 +
        int_acc   * 0.15 +
        avg_rouge * 0.20 +
        sec_acc   * 0.10 +
        cty_acc   * 0.03 +
        src_acc   * 0.02
    )

    return {
        "split"             : label,
        "n_samples"         : len(rows),
        "category_acc"      : round(cat_acc   * 100, 2),
        "aspect_acc"        : round(asp_acc   * 100, 2),
        "intent_acc"        : round(int_acc   * 100, 2),
        "country_acc"       : round(cty_acc   * 100, 2),
        "source_acc"        : round(src_acc   * 100, 2),
        "sector_acc"        : round(sec_acc   * 100, 2),
        "explanation_rougeL": round(avg_rouge * 100, 2),
        "OVERALL_SCORE_%"   : round(overall   * 100, 2),
    }


train_rows = [r for r in all_results if r["split"] == "train"]
eval_rows  = [r for r in all_results if r["split"] == "eval"]

train_scores = score_split(train_rows,  "TRAIN")
eval_scores  = score_split(eval_rows,   "EVAL")
all_scores   = score_split(all_results, "ALL")

def print_scores(s: dict):
    print(f"\n  ── {s['split']} SET  ({s['n_samples']} samples) ──")
    print(f"   Category    Accuracy  : {s['category_acc']:>7.2f} %   (weight 25%)")
    print(f"   Aspect      Accuracy  : {s['aspect_acc']:>7.2f} %   (weight 25%)")
    print(f"   Intent      Accuracy  : {s['intent_acc']:>7.2f} %   (weight 15%)")
    print(f"   Explanation ROUGE-L   : {s['explanation_rougeL']:>7.2f} %   (weight 20%)")
    print(f"   Sector      Accuracy  : {s['sector_acc']:>7.2f} %   (weight 10%)")
    print(f"   Country     Accuracy  : {s['country_acc']:>7.2f} %   (weight  3%)")
    print(f"   Source      Accuracy  : {s['source_acc']:>7.2f} %   (weight  2%)")
    print(f"   {'─'*42}")
    print(f"   ⭐ OVERALL SCORE      : {s['OVERALL_SCORE_%']:>7.2f} %")

print("\n" + "="*65)
print("              📊 ACCURACY REPORT")
print("="*65)
print_scores(train_scores)
print_scores(eval_scores)
print_scores(all_scores)
print("="*65)

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 14 ── Save everything to Excel in Drive
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
results_df = pd.DataFrame(all_results)
summary_df = pd.DataFrame([train_scores, eval_scores, all_scores])

os.makedirs(os.path.dirname(RESULTS_EXCEL_DRIVE), exist_ok=True)

with pd.ExcelWriter(RESULTS_EXCEL_DRIVE, engine="openpyxl") as writer:
    summary_df.to_excel(writer, sheet_name="Accuracy Summary", index=False)
    results_df.to_excel(writer, sheet_name="Per Sample",       index=False)

    for field in ["category", "aspect", "intent", "sector", "country", "source"]:
        breakdown = results_df[[
            "uni_id", "split",
            f"true_{field}", f"pred_{field}"
        ]].copy()
        breakdown["correct"] = (
            breakdown[f"true_{field}"].str.lower().str.strip() ==
            breakdown[f"pred_{field}"].str.lower().str.strip()
        )
        breakdown.to_excel(writer, sheet_name=f"{field.capitalize()} Detail", index=False)

print(f"\n✅ Full results saved → {RESULTS_EXCEL_DRIVE}")
print("\n🎉 ALL DONE!")
